In [1]:
import os
os.chdir('/Users/a589565/Desktop/Python Code/data')

In [2]:
import nltk

In [3]:
import csv

In [4]:
smsdata = open('SMSSpamCollection.txt','r')
csv_reader = csv.reader(smsdata,delimiter='\t')

smsdata_data = []
smsdata_labels = []

for line in csv_reader:
    smsdata_labels.append(line[0])
    smsdata_data.append(line[1])

smsdata.close()

In [5]:
smsdata

<_io.TextIOWrapper name='SMSSpamCollection.txt' mode='r' encoding='UTF-8'>

In [6]:
# Printing top 5 lines
for i in range(5):
    print (smsdata_data[i],smsdata_labels[i])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat... ham
Ok lar... Joking wif u oni... ham
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's spam
U dun say so early hor... U c already then say... ham
Nah I don't think he goes to usf, he lives around here though ham


In [7]:
# Printing Spam & Ham count
from collections import Counter
c = Counter( smsdata_labels )
print (c)

Counter({'ham': 4825, 'spam': 747})


In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
from nltk import pos_tag
from nltk.stem import PorterStemmer

In [9]:
def preprocessing(text):
    text2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in text]).split())

    tokens = [word for sent in nltk.sent_tokenize(text2) for word in
              nltk.word_tokenize(sent)]
    
    tokens = [word.lower() for word in tokens]
    
    stopwds = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopwds]
    
    tokens = [word for word in tokens if len(word)>=3]
    
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    tagged_corpus = pos_tag(tokens)    
    
    Noun_tags = ['NN','NNP','NNPS','NNS']
    Verb_tags = ['VB','VBD','VBG','VBN','VBP','VBZ']

    lemmatizer = WordNetLemmatizer()

    def prat_lemmatize(token,tag):
        if tag in Noun_tags:
            return lemmatizer.lemmatize(token,'n')
        elif tag in Verb_tags:
            return lemmatizer.lemmatize(token,'v')
        else:
            return lemmatizer.lemmatize(token,'n')
    
    pre_proc_text =  " ".join([prat_lemmatize(token,tag) for token,tag in tagged_corpus])             

    return pre_proc_text

In [10]:
smsdata_data_2 = []

for i in smsdata_data:
    smsdata_data_2.append(preprocessing(i))

In [11]:
smsdata_data_2

['jurong point crazi avail bugi great world buffet cine get amor wat',
 'lar joke wif oni',
 'free entri wkli comp win cup final tkt 21st may 2005 text 87121 receiv entri question std txt rate appli 08452810075over18',
 'dun say earli hor alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea £900 prize reward claim call 09061701461 claim code kl341 valid hour',
 'mobil month entitl updat latest colour mobil camera free call mobil updat free 08002986030',
 'gon home soon want talk stuff anymor tonight cri enough today',
 'six chanc win cash 100 000 pound txt csh11 send 87575 cost 150p day 6day tsandc appli repli info',
 'urgent week free membership £100 000 prize jackpot txt word claim 81010 www dbuk net lccltd pob

In [12]:
import numpy as np
trainset_size = int(round(len(smsdata_data)*0.70))
print ('The training set size for this classifier is ' + str(trainset_size) + '\n')
x_train = np.array([''.join(rec) for rec in smsdata_data[0:trainset_size]])
y_train = np.array([rec for rec in smsdata_labels[0:trainset_size]])
x_test = np.array([''.join(rec) for rec in smsdata_data[trainset_size+1:len(smsdata_data)]])
y_test = np.array([rec for rec in smsdata_labels[trainset_size+1:len(smsdata_labels)]])

The training set size for this classifier is 3900



In [13]:
x_train

array([ 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ...,
       'tells u 2 call 09066358152 to claim £5000 prize. U have 2 enter all ur mobile & personal details @ the prompts. Careful!',
       "No. Thank you. You've been wonderful",
       'Otherwise had part time job na-tuition..'],
      dtype='<U910')

In [14]:
# building TFIDF vectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1, 2),  stop_words='english', 
                             max_features= 4000,strip_accents='unicode',  norm='l2')

x_train_2 = vectorizer.fit_transform(x_train).todense()
x_test_2 = vectorizer.transform(x_test).todense()

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_train_2, y_train)

ytrain_nb_predicted = clf.predict(x_train_2)
ytest_nb_predicted = clf.predict(x_test_2)

from sklearn.metrics import classification_report,accuracy_score

In [15]:
clf = MultinomialNB()
%timeit clf.fit(x_train_2, y_train)

31.9 ms ± 913 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
print ("\nNaive Bayes - Train Confusion Matrix\n\n",pd.crosstab(y_train,ytrain_nb_predicted,rownames = ["Actuall"],colnames = ["Predicted"]))      
print ("\nNaive Bayes- Train accuracy",round(accuracy_score(y_train,ytrain_nb_predicted),3))
print ("\nNaive Bayes  - Train Classification Report\n",classification_report(y_train,ytrain_nb_predicted))

print ("\nNaive Bayes - Test Confusion Matrix\n\n",pd.crosstab(y_test,ytest_nb_predicted,rownames = ["Actuall"],colnames = ["Predicted"]))      
print ("\nNaive Bayes- Test accuracy",round(accuracy_score(y_test,ytest_nb_predicted),3))
print ("\nNaive Bayes  - Test Classification Report\n",classification_report(y_test,ytest_nb_predicted))


Naive Bayes - Train Confusion Matrix

 Predicted   ham  spam
Actuall              
ham        3381     0
spam         61   458

Naive Bayes- Train accuracy 0.984

Naive Bayes  - Train Classification Report
              precision    recall  f1-score   support

        ham       0.98      1.00      0.99      3381
       spam       1.00      0.88      0.94       519

avg / total       0.98      0.98      0.98      3900


Naive Bayes - Test Confusion Matrix

 Predicted   ham  spam
Actuall              
ham        1442     1
spam         46   182

Naive Bayes- Test accuracy 0.972

Naive Bayes  - Test Classification Report
              precision    recall  f1-score   support

        ham       0.97      1.00      0.98      1443
       spam       0.99      0.80      0.89       228

avg / total       0.97      0.97      0.97      1671



In [17]:
# printing top features 
feature_names = vectorizer.get_feature_names()
coefs = clf.coef_
intercept = clf.intercept_
coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))

print ("\n\nTop 10 features - both first & last\n")
n=10
top_n_coefs = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
for (coef_1, fn_1), (coef_2, fn_2) in top_n_coefs:
    print('\t%.4f\t%-15s\t\t%.4f\t%-15s' % (coef_1, fn_1, coef_2, fn_2))



Top 10 features - both first & last

	-8.7208	1st ur         		-5.6060	free           
	-8.7208	2go            		-5.8117	txt            
	-8.7208	2nd ur         		-6.0245	text           
	-8.7208	30             		-6.0764	claim          
	-8.7208	3d             		-6.0772	stop           
	-8.7208	3rd            		-6.1101	ur             
	-8.7208	3rd ur         		-6.1648	mobile         
	-8.7208	48             		-6.1670	reply          
	-8.7208	4a             		-6.2243	prize          
	-8.7208	4d             		-6.2600	www            


In [18]:
classification_report(y_test,ytest_nb_predicted)

'             precision    recall  f1-score   support\n\n        ham       0.97      1.00      0.98      1443\n       spam       0.99      0.80      0.89       228\n\navg / total       0.97      0.97      0.97      1671\n'

In [19]:
import pandas as pd
pd.crosstab(y_test,ytest_nb_predicted,rownames = ["Actuall"],colnames = ["Predicted"])

Predicted,ham,spam
Actuall,,
ham,1442,1
spam,46,182
